# Tutorial (5): how to define Hamiltonian and ansatz
This example shows how one can set a user-defined Hamiltonian and user-defined ansatz (for VQE).  
Quket takes a full advantage of OpenFermion library, and these user-defined objects should be in the OpenFermion format.

In [1]:
# import necessary modules
from openfermion import *
from quket import *
from quket.utils import *
import quket.config as cf
import numpy as np

mpi4py is not imported. no MPI.


[(Example 1) Hubbard Hamiltonian](#(Example-1)-Hubbard-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.1) Define Hamiltonian in Quket using option `hamiltonian`](#(1.1)-Define-Hamiltonian-in-Quket-using-option-hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.2) Run `fci2qubit()` to determine the ground state](#(1.2)-Run-fci2qubit()-to-determine-the-ground-state)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.3) Run `vqe()`](#(1.3)-Run-vqe())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.4) Transform the state to Jordan-Wigner's Particle-Hole representation](#(1.4)-Transform-the-state-to-Jordan-Wigner's-Particle-Hole-representation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.5) Tapering-off qubits](#(2.5)-Tapering-off-qubits)

[(Example2) Modified Kitaev Hamiltonian](#(Example2)-Modified-Kitaev-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.1) Define the modified Kitaev Hamiltonian](#(2.1)-Define-the-modified-Kitaev-Hamiltonian)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.2) Run `fci2qubit()` to determine the ground state](#(2.2)-Run-fci2qubit()-to-determine-the-ground-state)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.3) Manually define `pauli_list`](#(2.3)-Manually-define-pauli_list)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.4) Run `vqe()`](#(2.4)-Run-vqe())


## (Example 1) Hubbard Hamiltonian

 Define the Hubbard Hamiltonian using creation and annihilation operator

In [2]:
### Hubbard model as FermionOperator (OpenFermion part)
nx = 4
ny = 1
t = 1
U = 4
hubbard = fermi_hubbard(nx, ny, t, U,periodic=False)

### (1.1) Define Hamiltonian in Quket using option `hamiltonian`  
Here, we use the following options:
`ansatz`: Hamiltonian variational ansatz (HVA)  
`mapping`: Bravyi-Kitaev mapping  
`layers`: 2 layers of HVA  
`theta_guess`: Randomized VQE parameter guess  
`det`: Initial state is `|00000011>` (in the Jordan-Wigner representation: this will be transfromed to the Bravyi-Kitaev representation `|00000001>`)  
`maxiter`: Maximum iteration of VQE

In [3]:
Hub = create(hamiltonian=hubbard, ansatz='hva', mapping='bk', layers=2, theta_guess='random', det='|0011>', maxiter=200)

Simulation detail
Hamiltonian:
(4+0j) [] +
(-0.5+0j) [X0 Y1 Y2] +
(0.5+0j) [Y0 Y1 X2] +
(-1+0j) [Z0] +
(0.5+0j) [Z0 X1 Z3] +
(-1+0j) [Z0 Z1] +
(-0.5+0j) [X1 Z2] +
(1+0j) [Z1] +
(-0.5+0j) [Z1 X2 Y3 Y4 X5] +
(0.5+0j) [Z1 Y2 Y3 X4 X5] +
(-0.5+0j) [Z1 Z2 Y3 Y5] +
(-1+0j) [Z1 Z2 Z3] +
(1+0j) [Z1 Z3] +
(-1+0j) [Z2] +
(-0.5+0j) [X3 Z4 X5] +
(0.5+0j) [Z3 Z4 X5 Z7] +
(-1+0j) [Z3 Z5 Z6 Z7] +
(1+0j) [Z3 Z5 Z7] +
(-0.5+0j) [X4 Y5 Y6] +
(0.5+0j) [Y4 Y5 X6] +
(-1+0j) [Z4] +
(-1+0j) [Z4 Z5] +
(-0.5+0j) [X5 Z6] +
(1+0j) [Z5] +
(-1+0j) [Z6]
Initial state:
    Basis            Coef
| 00000001 > : +1.0000 +0.0000i



### (1.2) Run `fci2qubit()` to determine the ground state

In [4]:
Hub.fci2qubit(nroots=1)

Running OpenFemrion's QubitDavidson.
FCI in Qubits
(FCI state : E = -2.6249422715089907)
    Basis            Coef
| 00000011 > : +0.2074 +0.0000i
| 00000100 > : +0.1734 +0.0000i
| 00000110 > : -0.2074 -0.0000i
| 00001011 > : +0.2349 -0.0000i
| 00001100 > : +0.3083 -0.0000i
| 00010000 > : +0.1734 +0.0000i
| 00101011 > : +0.3083 -0.0000i
| 00101100 > : +0.3670 +0.0000i
| 00110000 > : +0.2074 +0.0000i
| 00111000 > : -0.3670 -0.0000i
| 00111010 > : -0.3083 +0.0000i
| 01001000 > : -0.3083 +0.0000i
| 01001010 > : -0.2349 +0.0000i
| 01100000 > : -0.2074 -0.0000i



### (1.3) Run `vqe()` 

In [5]:
Hub.vqe()

      1: E[hva] = +1.519975893562  <S**2> = +0.000000  Fidelity = 0.040982  Grad = 5.19e+00  CPU Time =    0.12068  (0.00 / step)
      2: E[hva] = +1.519973780719  <S**2> = +0.000000  Fidelity = 0.040983  Grad = 5.19e+00  CPU Time =    0.03235  (0.00 / step)
      3: E[hva] = +0.996838258845  <S**2> = +0.000000  Fidelity = 0.049486  Grad = 4.47e+00  CPU Time =    0.06186  (0.00 / step)
      4: E[hva] = +0.097549242111  <S**2> = +0.000000  Fidelity = 0.056717  Grad = 4.40e+00  CPU Time =    0.03000  (0.00 / step)
      5: E[hva] = -0.111841182173  <S**2> = +0.000000  Fidelity = 0.050103  Grad = 3.29e+00  CPU Time =    0.01599  (0.00 / step)
      6: E[hva] = -0.314591038829  <S**2> = +0.000000  Fidelity = 0.056612  Grad = 1.56e+00  CPU Time =    0.01580  (0.00 / step)
      7: E[hva] = -0.441594927366  <S**2> = +0.000000  Fidelity = 0.063734  Grad = 1.69e+00  CPU Time =    0.01724  (0.00 / step)
      8: E[hva] = -0.710203419306  <S**2> = +0.000000  Fidelity = 0.097280  Grad = 3.79e+0

### (1.4) Transform the state to Jordan-Wigner's Particle-Hole representation
For this purpose, `transform_state_bk2jw()` can be used.  
The opposite transformation is done with `transform_state_jw2bk()`.

In [6]:
jw_state = transform_state_bk2jw(Hub.state)
print_state(jw_state)

    Basis            Coef
| 00000110 > : -0.0797 -0.1926i
| 00001001 > : +0.0805 +0.1921i
| 00001100 > : +0.0668 +0.1599i
| 00010010 > : -0.1186 -0.2838i
| 00011000 > : -0.1419 -0.3401i
| 00100001 > : +0.1190 +0.2825i
| 00100100 > : +0.1416 +0.3395i
| 00110000 > : +0.0666 +0.1600i
| 01000010 > : -0.0909 -0.2179i
| 01001000 > : -0.1185 -0.2838i
| 01100000 > : -0.0797 -0.1917i
| 10000001 > : +0.0906 +0.2181i
| 10000100 > : +0.1186 +0.2822i
| 10010000 > : +0.0797 +0.1906i



Transform back to the bravyi-kitaev representation using `transform_state_jw2bk()`:

In [7]:
bk_state = transform_state_jw2bk(jw_state)
print_state(bk_state)
print("They are the same state", inner_product(bk_state, Hub.state))

    Basis            Coef
| 00000011 > : +0.0805 +0.1921i
| 00000100 > : +0.0668 +0.1599i
| 00000110 > : -0.0797 -0.1926i
| 00001011 > : +0.0906 +0.2181i
| 00001100 > : +0.1186 +0.2822i
| 00010000 > : +0.0666 +0.1600i
| 00101011 > : +0.1190 +0.2825i
| 00101100 > : +0.1416 +0.3395i
| 00110000 > : +0.0797 +0.1906i
| 00111000 > : -0.1419 -0.3401i
| 00111010 > : -0.1186 -0.2838i
| 01001000 > : -0.1185 -0.2838i
| 01001010 > : -0.0909 -0.2179i
| 01100000 > : -0.0797 -0.1917i

They are the same state (0.9999999999999988+0j)


### (2.5) Tapering-off qubits
Symmetry-search and tapering-off qubits can be performed for user-defined Hamiltonians.

In [8]:
Hub.taper_off()

Tapering-Off Results:
List of Tapered-off Qubits:  [0, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6]
Qubit: 7    Tau: 1.0 [Z7]

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.


In [9]:
print_state(Hub.state)

   Basis           Coef
| 000001 > : +0.0805 +0.1921i
| 000010 > : -0.0668 -0.1599i
| 000011 > : +0.0797 +0.1926i
| 000101 > : +0.0906 +0.2181i
| 000110 > : -0.1186 -0.2822i
| 001000 > : -0.0666 -0.1600i
| 010101 > : +0.1190 +0.2825i
| 010110 > : -0.1416 -0.3395i
| 011000 > : -0.0797 -0.1906i
| 011100 > : +0.1419 +0.3401i
| 011101 > : +0.1186 +0.2838i
| 100100 > : +0.1185 +0.2838i
| 100101 > : +0.0909 +0.2179i
| 110000 > : +0.0797 +0.1917i



In [10]:
print("Qubits in the tapered-mapping", Hub.n_qubits)
print("Qubits in the original mapping", Hub._n_qubits)

Qubits in the tapered-mapping 6
Qubits in the original mapping 8


In [11]:
Hub.vqe()

      1: E[hva] = -2.624859103249  <S**2> = +0.000000  Fidelity = 0.999979  Grad = 1.87e-03  CPU Time =    0.04368  (0.00 / step)
      2: E[hva] = -2.624859241645  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 1.24e-03  CPU Time =    0.01648  (0.00 / step)
      3: E[hva] = -2.624859491112  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 9.99e-04  CPU Time =    0.01578  (0.00 / step)
      4: E[hva] = -2.624859606561  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 8.20e-04  CPU Time =    0.01540  (0.00 / step)
      5: E[hva] = -2.624859698816  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 1.31e-03  CPU Time =    0.01560  (0.00 / step)
      6: E[hva] = -2.624859801468  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 7.51e-04  CPU Time =    0.01380  (0.00 / step)
      7: E[hva] = -2.624859918602  <S**2> = +0.000000  Fidelity = 0.999980  Grad = 8.96e-04  CPU Time =    0.01482  (0.00 / step)
      8: E[hva] = -2.624860116990  <S**2> = +0.000000  Fidelity = 0.999981  Grad = 1.29e-0

## (Example2) Modified Kitaev Hamiltonian
This sample performs VQE with a modified Kitaev Hamiltonian and Hamiltonian Variational Ansatz of arXiv:2108.13375.

```               
  0           3 
  Z \   X   / Z
      1 - 2  
    Y |   |  Y
      6 - 4  
    /   X   \
  7 Z      Z  5 
                

  Eg = -4.0100  (TCz in Table I)
  Eg = -4.2476  (TCz+h in Table I)
```



### (2.1) Define the modified Kitaev Hamiltonian

In [12]:
### Define the modified Kitaev Hamiltonian ###
### Parameters for TCz+h

Jx = -0.1
Jy = Jx
Jz = -1
hx = 0.05/np.sqrt(3)
hy = hx 
hz = hx 

### User-defined Hamiltonian in the OpenFermion format.
hamiltonian =f"{Jx} [X1 X2] + {Jx} [X4 X6] \
             + {Jy} [Y1 Y6] + {Jy} [Y2 Y4] \
             + {Jz} [Z0 Z1] + {Jz} [Z2 Z3] + {Jz} [Z4 Z5] + {Jz} [Z6 Z7] \
             + {hx} [X0] + {hx} [X1] + {hx} [X2] + {hx} [X3] \
             + {hx} [X4] + {hx} [X5] + {hx} [X6] + {hx} [X7] \
             + {hy} [Y0] + {hy} [Y1] + {hy} [Y2] + {hy} [Y3] \
             + {hy} [Y4] + {hy} [Y5] + {hy} [Y6] + {hy} [Y7] \
             + {hz} [Z0] + {hz} [Z1] + {hz} [Z2] + {hz} [Z3] \
             + {hz} [Z4] + {hz} [Z5] + {hz} [Z6] + {hz} [Z7] "

In [13]:
### Either of the followings works
BK = create(hamiltonian=QubitOperator(hamiltonian))
#BK = create(hamiltonian=hamiltonian)

Initial state is not supplied, so we use |0>.
You may specify the initial state by one of the following ways:

  Q.set(det='|0011>')
for product state,
  Q.set(det='0.5 * |0011> + 0.5 * |0101>')
for entangled state, and
  Q.init_state = QuantumState(Q.n_qubits)
  ...
to directly subsititute user-defined QuantumState to init_state.

Simulation detail
Hamiltonian:
0.02886751345948129 [X0] +
0.02886751345948129 [Y0] +
0.02886751345948129 [Z0] +
-1.0 [Z0 Z1] +
0.02886751345948129 [X1] +
-0.1 [X1 X2] +
0.02886751345948129 [Y1] +
-0.1 [Y1 Y6] +
0.02886751345948129 [Z1] +
0.02886751345948129 [X2] +
0.02886751345948129 [Y2] +
-0.1 [Y2 Y4] +
0.02886751345948129 [Z2] +
-1.0 [Z2 Z3] +
0.02886751345948129 [X3] +
0.02886751345948129 [Y3] +
0.02886751345948129 [Z3] +
0.02886751345948129 [X4] +
-0.1 [X4 X6] +
0.02886751345948129 [Y4] +
0.02886751345948129 [Z4] +
-1.0 [Z4 Z5] +
0.02886751345948129 [X5] +
0.02886751345948129 [Y5] +
0.02886751345948129 [Z5] +
0.02886751345948129 [X6] +
0.028867513459481

### (2.2) Run `fci2qubit()` to determine the ground state

In [14]:
BK.fci2qubit()

Running OpenFemrion's QubitDavidson.
FCI in Qubits
(FCI state : E = -4.24757473769705)
    Basis            Coef
| 11111111 > : -0.9965 +0.0000i



### (2.3) Manually define `pauli_list`
If only `hamiltonian` is defined, `Quket` complains that there is no variational circuit.  
We can provide either `ansatz` or `pauli_list` to construct a variational circuit.  
Here, let us define `pauli_list` manually, which can be done by using option `pauli_list`.  
  
In a `QuketData` instance, the attribute `QuketData.pauli_list` is a list of `QubitOperator`s.  
When specifying `pauli_list` as an option of `set()` or `create()`, it can be either `QubitOperator` or `str` (which will be converted).

In [15]:
# As an example, we take HVA but define it manually.
pauli_list =['[X1 X2]', '[X4 X6]', '[Y1 Y6]', '[Y2 Y4]', 
             '[Z0 Z1]', '[Z2 Z3]', '[Z4 Z5]', '[Z6 Z7]', 
             '[X0]', '[X1]', '[X2]', '[X3]',
             '[X4]', '[X5]', '[X6]', '[X7]',
             '[Y0]', '[Y1]', '[Y2]', '[Y3]',
             '[Y4]', '[Y5]', '[Y6]', '[Y7]',
             '[Z0]', '[Z1]', '[Z2]', '[Z3]',
             '[Z4]', '[Z5]', '[Z6]', '[Z7]',]
# Additionally let us set other options.
BK.set(pauli_list=pauli_list, layers=1, theta_guess="random", det="|11111111>")
BK.initialize()

Simulation detail
Hamiltonian:
0.02886751345948129 [X0] +
0.02886751345948129 [Y0] +
0.02886751345948129 [Z0] +
-1.0 [Z0 Z1] +
0.02886751345948129 [X1] +
-0.1 [X1 X2] +
0.02886751345948129 [Y1] +
-0.1 [Y1 Y6] +
0.02886751345948129 [Z1] +
0.02886751345948129 [X2] +
0.02886751345948129 [Y2] +
-0.1 [Y2 Y4] +
0.02886751345948129 [Z2] +
-1.0 [Z2 Z3] +
0.02886751345948129 [X3] +
0.02886751345948129 [Y3] +
0.02886751345948129 [Z3] +
0.02886751345948129 [X4] +
-0.1 [X4 X6] +
0.02886751345948129 [Y4] +
0.02886751345948129 [Z4] +
-1.0 [Z4 Z5] +
0.02886751345948129 [X5] +
0.02886751345948129 [Y5] +
0.02886751345948129 [Z5] +
0.02886751345948129 [X6] +
0.02886751345948129 [Y6] +
0.02886751345948129 [Z6] +
-1.0 [Z6 Z7] +
0.02886751345948129 [X7] +
0.02886751345948129 [Y7] +
0.02886751345948129 [Z7]
Initial state:
    Basis            Coef
| 11111111 > : +1.0000 +0.0000i

user-defined pauli_list found.


#### Check `pauli_list` attribute.

In [16]:
BK.pauli_list

[1.0 [X1 X2],
 1.0 [X4 X6],
 1.0 [Y1 Y6],
 1.0 [Y2 Y4],
 1.0 [Z0 Z1],
 1.0 [Z2 Z3],
 1.0 [Z4 Z5],
 1.0 [Z6 Z7],
 1.0 [X0],
 1.0 [X1],
 1.0 [X2],
 1.0 [X3],
 1.0 [X4],
 1.0 [X5],
 1.0 [X6],
 1.0 [X7],
 1.0 [Y0],
 1.0 [Y1],
 1.0 [Y2],
 1.0 [Y3],
 1.0 [Y4],
 1.0 [Y5],
 1.0 [Y6],
 1.0 [Y7],
 1.0 [Z0],
 1.0 [Z1],
 1.0 [Z2],
 1.0 [Z3],
 1.0 [Z4],
 1.0 [Z5],
 1.0 [Z6],
 1.0 [Z7]]

### (2.4) Run `vqe()`

In [17]:
BK.vqe()

      1: E[user-defined] = -4.237671956560  <S**2> = +0.000000  Fidelity = 0.996665  Grad = 2.72e-02  CPU Time =    0.03718  (0.00 / step)
      2: E[user-defined] = -4.237765746208  <S**2> = +0.000000  Fidelity = 0.996736  Grad = 6.63e-03  CPU Time =    0.01040  (0.00 / step)
      3: E[user-defined] = -4.237766345596  <S**2> = +0.000000  Fidelity = 0.996736  Grad = 5.71e-03  CPU Time =    0.00970  (0.00 / step)
      4: E[user-defined] = -4.237768401639  <S**2> = +0.000000  Fidelity = 0.996736  Grad = 9.00e-05  CPU Time =    0.00946  (0.00 / step)
      5: E[user-defined] = -4.237815472174  <S**2> = +0.000000  Fidelity = 0.996547  Grad = 8.74e-02  CPU Time =    0.08808  (0.00 / step)
      6: E[user-defined] = -4.237815591531  <S**2> = +0.000000  Fidelity = 0.996542  Grad = 8.86e-02  CPU Time =    0.01927  (0.00 / step)
      7: E[user-defined] = -4.237830311623  <S**2> = +0.000000  Fidelity = 0.996523  Grad = 9.33e-02  CPU Time =    0.00953  (0.00 / step)
      8: E[user-defined] = 

#### There does not seem to be any symmetry in the modified Kitaev Hamiltonian, 
#### and hence `taper_off()` does nothing.

In [18]:
BK.taper_off()

Tapering-Off Results:
List of Tapered-off Qubits:  None

No qubits to be tapered-off. No transformation done
